In [1]:
!pip install -q transformers

     |████████████████████████████████| 2.8 MB 9.1 MB/s 
     |████████████████████████████████| 636 kB 48.7 MB/s 
     |████████████████████████████████| 895 kB 35.3 MB/s 
     |████████████████████████████████| 3.3 MB 39.1 MB/s 
     |████████████████████████████████| 50 kB 7.1 MB/s 


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
%matplotlib inline
import os
import torch
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn import metrics
from sklearn.metrics import classification_report
import re
from sklearn.model_selection import train_test_split
from transformers import BertTokenizer
from transformers import AutoTokenizer, AutoModel
from transformers import BertForSequenceClassification, AdamW, BertConfig
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler

import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, random_split
from torch.utils.data.sampler import SubsetRandomSampler
import transformers
from transformers import RobertaTokenizer, BertTokenizer, RobertaModel, BertModel, AdamW# get_linear_schedule_with_warmup
from transformers import get_linear_schedule_with_warmup
import time

!cp drive/MyDrive/Colab\ Notebooks/MSc-Individual-Project/utils.py .
from utils import *
!cp drive/MyDrive/Colab\ Notebooks/MSc-Individual-Project/Custom_Dataset_Class.py .
from Custom_Dataset_Class import CustomDataset
!cp drive/MyDrive/Colab\ Notebooks/MSc-Individual-Project/pytorchtools.py .
from pytorchtools import EarlyStopping
#from Bert_Classification import Bert_Classification_Model
#from RoBERT import RoBERT_Model

#from BERT_Hierarchical import BERT_Hierarchical_Model
import warnings
warnings.filterwarnings("ignore")

from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.preprocessing import LabelBinarizer

In [4]:
import torch
# If there's a GPU available...
if torch.cuda.is_available():    

    # Tell PyTorch to use the GPU.    
    device = torch.device("cuda")

    print('There are %d GPU(s) available.' % torch.cuda.device_count())

    print('We will use the GPU:', torch.cuda.get_device_name(0))

# If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")


There are 1 GPU(s) available.
We will use the GPU: Tesla P100-PCIE-16GB


In [5]:
np.random.seed(123)
torch.manual_seed(123)
torch.cuda.manual_seed_all(123)

In [96]:
#change to where you store mimic3 data
MIMIC_3_DIR = '/content/drive/MyDrive/Colab Notebooks/MSc-Individual-Project/datasets'

train_df = pd.read_csv('%s/train_10.csv' % MIMIC_3_DIR)
eval_df = pd.read_csv('%s/dev_10.csv' % MIMIC_3_DIR)
test_df = pd.read_csv('%s/test_10.csv' % MIMIC_3_DIR)

train_df.head()

,SUBJECT_ID,HADM_ID,CATEGORY,TEXT,LABELS,length
0,17341,151110,Nursing/other,rsbi,584.9;427.31,1
1,61638,103816,Nursing,title,414.01,1
2,61638,103816,General,title,414.01,1
3,23706,186321,Nursing/other,npn,401.9;428.0;530.81,1
4,55265,191108,General,title,530.81;584.9;427.31,1


In [97]:
full_df = pd.concat([train_df, eval_df, test_df], ignore_index=True)

In [98]:
full_df['TEXT'] = 'Note' + full_df['CATEGORY'] + ' ' + full_df['TEXT'] 

In [99]:
full_df

,SUBJECT_ID,HADM_ID,CATEGORY,TEXT,LABELS,length
0,17341,151110,Nursing/other,NoteNursing/other rsbi,584.9;427.31,1
1,61638,103816,Nursing,NoteNursing title,414.01,1
2,61638,103816,General,NoteGeneral title,414.01,1
3,23706,186321,Nursing/other,NoteNursing/other npn,401.9;428.0;530.81,1
4,55265,191108,General,NoteGeneral title,530.81;584.9;427.31,1
...,...,...,...,...,...,...
294460,97158,152158,Discharge summary,NoteDischarge summary admission date discharge...,401.9;518.81,4644
294461,99650,199859,Discharge summary,NoteDischarge summary admission date discharge...,584.9;518.81;427.31;414.01;599.0;428.0,5126
294462,93623,187232,Discharge summary,NoteDischarge summary admission date discharge...,401.9;272.4,5171
294463,96260,110058,Discharge summary,NoteDischarge summary admission date discharge...,518.81;599.0,5173


In [100]:
# split labels by ";", then convert to list
def split_lab (x):
    #print(x)
    return x.split(";")

full_df['LABELS'] = full_df['LABELS'].apply(split_lab)
#full_df['TEXT'] = full_df['TEXT'].apply(split_lab)

full_df.head()

,SUBJECT_ID,HADM_ID,CATEGORY,TEXT,LABELS,length
0,17341,151110,Nursing/other,NoteNursing/other rsbi,"[584.9, 427.31]",1
1,61638,103816,Nursing,NoteNursing title,[414.01],1
2,61638,103816,General,NoteGeneral title,[414.01],1
3,23706,186321,Nursing/other,NoteNursing/other npn,"[401.9, 428.0, 530.81]",1
4,55265,191108,General,NoteGeneral title,"[530.81, 584.9, 427.31]",1


In [101]:
#load multi label binarizer for one-hot encoding
mlb = MultiLabelBinarizer(sparse_output=True)

#labels_onehot = mlb.fit_transform(train_df.pop('LABELS'))
#labels_onehot[0][1]

In [102]:
#change label to one-hot encoding per code
full_df = full_df.join(
            pd.DataFrame.sparse.from_spmatrix(
                mlb.fit_transform(full_df.pop('LABELS')),
                columns=mlb.classes_))

full_df

,SUBJECT_ID,HADM_ID,CATEGORY,TEXT,length,250.00,272.4,401.9,414.01,427.31,428.0,518.81,530.81,584.9,599.0
0,17341,151110,Nursing/other,NoteNursing/other rsbi,1,0,0,0,0,1,0,0,0,1,0
1,61638,103816,Nursing,NoteNursing title,1,0,0,0,1,0,0,0,0,0,0
2,61638,103816,General,NoteGeneral title,1,0,0,0,1,0,0,0,0,0,0
3,23706,186321,Nursing/other,NoteNursing/other npn,1,0,0,1,0,0,1,0,1,0,0
4,55265,191108,General,NoteGeneral title,1,0,0,0,0,1,0,0,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
294460,97158,152158,Discharge summary,NoteDischarge summary admission date discharge...,4644,0,0,1,0,0,0,1,0,0,0
294461,99650,199859,Discharge summary,NoteDischarge summary admission date discharge...,5126,0,0,0,1,1,1,1,0,1,1
294462,93623,187232,Discharge summary,NoteDischarge summary admission date discharge...,5171,0,1,1,0,0,0,0,0,0,0
294463,96260,110058,Discharge summary,NoteDischarge summary admission date discharge...,5173,0,0,0,0,0,0,1,0,0,1


In [103]:
# Convert columns to list of one hot encoding
icd_classes_50 = mlb.classes_

full_df['labels'] = full_df[icd_classes_50].values.tolist()
#train_df.sort_values(['length'], ascending=False, inplace=True)
full_df.head()


,SUBJECT_ID,HADM_ID,CATEGORY,TEXT,length,250.00,272.4,401.9,414.01,427.31,428.0,518.81,530.81,584.9,599.0,labels
0,17341,151110,Nursing/other,NoteNursing/other rsbi,1,0,0,0,0,1,0,0,0,1,0,"[0, 0, 0, 0, 1, 0, 0, 0, 1, 0]"
1,61638,103816,Nursing,NoteNursing title,1,0,0,0,1,0,0,0,0,0,0,"[0, 0, 0, 1, 0, 0, 0, 0, 0, 0]"
2,61638,103816,General,NoteGeneral title,1,0,0,0,1,0,0,0,0,0,0,"[0, 0, 0, 1, 0, 0, 0, 0, 0, 0]"
3,23706,186321,Nursing/other,NoteNursing/other npn,1,0,0,1,0,0,1,0,1,0,0,"[0, 0, 1, 0, 0, 1, 0, 1, 0, 0]"
4,55265,191108,General,NoteGeneral title,1,0,0,0,0,1,0,0,1,1,0,"[0, 0, 0, 0, 1, 0, 0, 1, 1, 0]"


In [104]:
full_df = full_df.drop(full_df[full_df['length']<300].index)

In [105]:
full_df

,SUBJECT_ID,HADM_ID,CATEGORY,TEXT,length,250.00,272.4,401.9,414.01,427.31,428.0,518.81,530.81,584.9,599.0,labels
155040,65582,194117,Nursing,NoteNursing name8 md md h p yo male with strep...,300,0,0,0,0,0,0,1,0,0,1,"[0, 0, 0, 0, 0, 0, 1, 0, 0, 1]"
155041,70924,119848,Nursing,NoteNursing 42f with chronic alcoholic pancrea...,300,0,0,0,0,0,0,1,0,1,1,"[0, 0, 0, 0, 0, 0, 1, 0, 1, 1]"
155042,25139,180782,Radiology,NoteRadiology am picc line placment sch clip c...,300,0,0,0,0,0,0,0,0,1,0,"[0, 0, 0, 0, 0, 0, 0, 0, 1, 0]"
155043,13494,164195,Radiology,NoteRadiology pm ct abdomen w o contrast ct pe...,300,0,0,0,0,0,0,1,0,0,1,"[0, 0, 0, 0, 0, 0, 1, 0, 0, 1]"
155044,30275,152118,Radiology,NoteRadiology pm perc g g j tube plmt clip cli...,300,0,0,0,0,0,0,1,0,0,1,"[0, 0, 0, 0, 0, 0, 1, 0, 0, 1]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
294460,97158,152158,Discharge summary,NoteDischarge summary admission date discharge...,4644,0,0,1,0,0,0,1,0,0,0,"[0, 0, 1, 0, 0, 0, 1, 0, 0, 0]"
294461,99650,199859,Discharge summary,NoteDischarge summary admission date discharge...,5126,0,0,0,1,1,1,1,0,1,1,"[0, 0, 0, 1, 1, 1, 1, 0, 1, 1]"
294462,93623,187232,Discharge summary,NoteDischarge summary admission date discharge...,5171,0,1,1,0,0,0,0,0,0,0,"[0, 1, 1, 0, 0, 0, 0, 0, 0, 0]"
294463,96260,110058,Discharge summary,NoteDischarge summary admission date discharge...,5173,0,0,0,0,0,0,1,0,0,1,"[0, 0, 0, 0, 0, 0, 1, 0, 0, 1]"


In [106]:
train_df, test_df = train_test_split(full_df, test_size=0.2)

In [107]:
train_df, dev_df = train_test_split(train_df, test_size=0.2)

In [108]:
train_df

,SUBJECT_ID,HADM_ID,CATEGORY,TEXT,length,250.00,272.4,401.9,414.01,427.31,428.0,518.81,530.81,584.9,599.0,labels
276929,96962,163165,Radiology,NoteRadiology am ct head w o contrast clip cli...,312,0,0,1,0,1,0,1,1,0,1,"[0, 0, 1, 0, 1, 0, 1, 1, 0, 1]"
160250,15106,195209,Nursing/other,NoteNursing/other update d pt pod from cabg x ...,326,1,0,1,1,0,1,0,0,0,0,"[1, 0, 1, 1, 0, 1, 0, 0, 0, 0]"
241795,83607,156758,Nursing,NoteNursing yo f with esrd on hd hbv recent pr...,639,0,0,0,0,0,0,1,0,0,0,"[0, 0, 0, 0, 0, 0, 1, 0, 0, 0]"
188047,77028,182444,Rehab Services,NoteRehab Services attending physician name10 ...,629,1,1,1,1,0,0,0,0,0,0,"[1, 1, 1, 1, 0, 0, 0, 0, 0, 0]"
164592,21592,198847,Nursing/other,NoteNursing/other shift note age over yr male ...,352,0,0,0,0,1,0,1,0,1,1,"[0, 0, 0, 0, 1, 0, 1, 0, 1, 1]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
172827,7888,106348,Nursing/other,NoteNursing/other nursing progress note please...,417,1,0,1,0,1,0,0,0,0,1,"[1, 0, 1, 0, 1, 0, 0, 0, 0, 1]"
284862,92243,158392,Radiology,NoteRadiology am ct abdomen w contrast ct pelv...,548,0,0,0,0,0,0,1,0,0,0,"[0, 0, 0, 0, 0, 0, 1, 0, 0, 0]"
293473,95204,133383,Discharge summary,NoteDischarge summary admission date discharge...,1614,0,0,1,0,0,0,0,0,1,0,"[0, 0, 1, 0, 0, 0, 0, 0, 1, 0]"
238776,83555,102515,Nursing,NoteNursing year old male w myxomatous mitral ...,507,0,0,0,1,1,0,1,0,0,0,"[0, 0, 0, 1, 1, 0, 1, 0, 0, 0]"


In [109]:
train_df.HADM_ID.unique().shape

(8424,)

In [30]:
train_df.sort_values(['length'], inplace=True)
dev_df.sort_values(['length'], inplace=True)
test_df.sort_values(['length'], inplace=True)


In [31]:
#convert into 2 columns dataframe
train_df = pd.DataFrame(train_df, columns=['TEXT', 'labels'])
train_df.columns=['text', 'labels']
train_df.head()

dev_df = pd.DataFrame(dev_df, columns=['HADM_ID', 'TEXT', 'labels'])
dev_df.columns=['id', 'text', 'labels']
dev_df.head()

test_df = pd.DataFrame(test_df, columns=['HADM_ID', 'TEXT', 'labels'])
test_df.columns=['id', 'text', 'labels']
test_df.head()

,id,text,labels
155048,151838,NoteEcho patient test information indication c...,"[0, 0, 0, 1, 0, 1, 0, 0, 1, 0]"
155165,135559,NoteRadiology pm pelvis limited pelvis u s tra...,"[0, 0, 0, 0, 0, 0, 1, 0, 0, 0]"
155201,132213,NoteNursing/other pmicu nursing progress 7a 7p...,"[0, 0, 0, 0, 0, 0, 0, 1, 0, 0]"
155126,183196,NoteRadiology pm ct chest w contrast clip clip...,"[1, 0, 0, 1, 0, 0, 0, 0, 0, 0]"
155195,159509,NoteEcho patient test information indication l...,"[1, 0, 0, 0, 1, 1, 0, 0, 0, 0]"


In [32]:
train_df.reset_index(drop=True, inplace=True)
dev_df.reset_index(drop=True, inplace=True)
test_df.reset_index(drop=True, inplace=True)
test_df.head()

,id,text,labels
0,151838,NoteEcho patient test information indication c...,"[0, 0, 0, 1, 0, 1, 0, 0, 1, 0]"
1,135559,NoteRadiology pm pelvis limited pelvis u s tra...,"[0, 0, 0, 0, 0, 0, 1, 0, 0, 0]"
2,132213,NoteNursing/other pmicu nursing progress 7a 7p...,"[0, 0, 0, 0, 0, 0, 0, 1, 0, 0]"
3,183196,NoteRadiology pm ct chest w contrast clip clip...,"[1, 0, 0, 1, 0, 0, 0, 0, 0, 0]"
4,159509,NoteEcho patient test information indication l...,"[1, 0, 0, 0, 1, 1, 0, 0, 0, 0]"


In [33]:
# Creating the customized model, by adding a drop out and a dense layer on top of distil bert to get the final output for the model.

class BERTClass(torch.nn.Module):
    def __init__(self):
        super(BERTClass, self).__init__()
        '''
            Load Pretrained model here
            Use return_dict=False for compatibility for 4.x

        '''
        self.l1 = transformers.AutoModel.from_pretrained("emilyalsentzer/Bio_ClinicalBERT", return_dict=False)
        #self.l1 = transformers.BertModel.from_pretrained('bert-base-uncased', return_dict=False)


        self.l2 = torch.nn.Dropout(0.3)

        self.l3 = torch.nn.Linear(768, 10)

    def forward(self, ids, mask, token_type_ids):
#        print("ids: ", ids.size(), "mask: ", mask.size(), "token type ids: ", token_type_ids.size())
        _, output_1= self.l1(ids, attention_mask = mask, token_type_ids = token_type_ids)
        output_2 = self.l2(output_1)
        output = self.l3(output_2)
        return output

model = BERTClass()
model.to(device)

Downloading:   0%|          | 0.00/385 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/436M [00:00<?, ?B/s]

Some weights of the model checkpoint at emilyalsentzer/Bio_ClinicalBERT were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


BERTClass(
  (l1): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(28996, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    

In [34]:
# Defining some key variables to configure model training
MAX_LEN = 512
TRAIN_BATCH_SIZE = 12
VALID_BATCH_SIZE = 8
TEST_BATCH_SIZE = 8
EPOCHS = 5
LEARNING_RATE = 3e-05

#set tokenizer
tokenizer = AutoTokenizer.from_pretrained("emilyalsentzer/Bio_ClinicalBERT")

#custom dataset for BERT class
class CustomDataset(Dataset):

    def __init__(self, dataframe, tokenizer, max_len):
        
        '''
            set text as training data
            set labels as targets
        '''
        self.tokenizer = tokenizer
        self.data = dataframe
        self.text = dataframe.text
        self.targets = self.data.labels
        self.max_len = max_len

    def __len__(self):
        return len(self.text)

    def __getitem__(self, index):
        text = str(self.text[index])
        text = " ".join(text.split())

        inputs = self.tokenizer.encode_plus(
            text,
            None,
            add_special_tokens=True,
            max_length=self.max_len,
            pad_to_max_length=True,
            return_token_type_ids=True
        )
        ids = inputs['input_ids']
        mask = inputs['attention_mask']
        token_type_ids = inputs["token_type_ids"]


        return {
            'ids': torch.tensor(ids, dtype=torch.long),
            'mask': torch.tensor(mask, dtype=torch.long),
            'token_type_ids': torch.tensor(token_type_ids, dtype=torch.long),
            'targets': torch.tensor(self.targets[index], dtype=torch.float)
        }



Downloading:   0%|          | 0.00/213k [00:00<?, ?B/s]

In [35]:
#load df to dataset

training_set = CustomDataset(train_df, tokenizer, MAX_LEN)
dev_set = CustomDataset(dev_df, tokenizer, MAX_LEN)
testing_set = CustomDataset(test_df, tokenizer, MAX_LEN)

In [36]:
#data loader
train_params = {'batch_size': TRAIN_BATCH_SIZE,
                'shuffle': False
                }


dev_params = {'batch_size': TEST_BATCH_SIZE,
                'shuffle': False
                }

test_params = {'batch_size': TEST_BATCH_SIZE,
                'shuffle': False
                }

training_loader = DataLoader(training_set, **train_params)
dev_loader = DataLoader(dev_set, **dev_params)
testing_loader = DataLoader(testing_set, **test_params)

In [37]:
#loss function
def loss_fn(outputs, targets):
    return torch.nn.BCEWithLogitsLoss()(outputs, targets)

#optimizer
optimizer = torch.optim.Adam(params=model.parameters(), lr=LEARNING_RATE)

In [38]:
def train(epoch):
    model.train()
    for _,data in enumerate(training_loader, 0):
        ids = data['ids'].to(device, dtype = torch.long)
        mask = data['mask'].to(device, dtype = torch.long)
        token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)
        targets = data['targets'].to(device, dtype = torch.float)

        outputs = model(ids, mask, token_type_ids)

        optimizer.zero_grad()
        loss = loss_fn(outputs, targets)
        loss.backward()
        optimizer.step()
        
    print(f'Epoch: {epoch}, Training Loss:  {loss.item()}')

In [39]:
# Evaluate the model

def validation(epoch):
    model.eval()
    fin_targets=[]
    fin_outputs=[]
    losses=[]
    with torch.no_grad():
        for _, data in enumerate(dev_loader, 0):
            ids = data['ids'].to(device, dtype = torch.long)
            mask = data['mask'].to(device, dtype = torch.long)
            token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)
            targets = data['targets'].to(device, dtype = torch.float)
            outputs = model(ids, mask, token_type_ids)
            loss = loss_fn(outputs, targets)
            losses.append(loss.item())

            fin_targets.extend(targets.cpu().detach().numpy().tolist())
            fin_outputs.extend(torch.sigmoid(outputs).cpu().detach().numpy().tolist())
    print(f'Epoch: {epoch}, Validation Loss:  {np.mean(losses):.2f}')
    return fin_outputs, fin_targets, losses

In [41]:
start_epoch=0
DIR = '/content/drive/MyDrive/Colab Notebooks/MSc-Individual-Project/'
resume = True     
if resume:
    if os.path.isfile(f"%s/models/model_allnotes10withcat_epoch{start_epoch}.pth" % DIR):
        print("Resume from checkpoint...")
        checkpoint = torch.load(f"%s/models/model_allnotes10withcat_epoch{start_epoch}.pth" % DIR)
        model.load_state_dict(checkpoint['model_state_dict'])
        optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
        initepoch = checkpoint['epoch']
        print("====>loaded checkpoint (epoch{})".format(checkpoint['epoch']))
    else:
        print("====>no checkpoint found.")
        initepoch = 0

#patience = 3
#early_stopping = EarlyStopping(patience, verbose=True)


for epoch in tqdm(range(EPOCHS)):
    train(epoch)
    validation(epoch)

    if (epoch+start_epoch+1)%5 == 0:
        checkpoint = {"model_state_dict": model.state_dict(),
                      "optimizer_state_dict": optimizer.state_dict(),
                      "epoch": epoch+start_epoch+1}
        path_checkpoint = f"%s/models/model_allnotes10withcat_epoch{epoch+start_epoch+1}.pth" % DIR
        torch.save(checkpoint, path_checkpoint)

#

====>no checkpoint found.


  0%|          | 0/5 [00:00<?, ?it/s]

Epoch: 0, Training Loss:  0.3196175992488861


 20%|██        | 1/5 [58:30<3:54:01, 3510.41s/it]

Epoch: 0, Validation Loss:  0.42
Epoch: 1, Training Loss:  0.2814151644706726


...

 60%|██████    | 3/5 [2:57:57<1:58:53, 3566.63s/it]

Epoch: 2, Validation Loss:  0.31
Epoch: 3, Training Loss:  0.2535877823829651


 80%|████████  | 4/5 [3:57:28<59:28, 3568.24s/it]  

Epoch: 3, Validation Loss:  0.29
Epoch: 4, Training Loss:  0.24194730818271637
Epoch: 4, Validation Loss:  0.28


100%|██████████| 5/5 [4:57:02<00:00, 3564.58s/it]


In [ ]:
DIR = '/content/drive/MyDrive/Colab Notebooks/MSc-Individual-Project/'

checkpoint = torch.load(f"%s/models/model_allnotes10withcat_epoch5.pth" % DIR)
model.load_state_dict(checkpoint['model_state_dict'])



<All keys matched successfully>

In [42]:
# Evaluate the model

def evaluation():
    model.eval()

    fin_targets=[]
    fin_outputs=[]
    losses=[]
    with torch.no_grad():
        for _, data in enumerate(dev_loader, 0):
            ids = data['ids'].to(device, dtype = torch.long)
            mask = data['mask'].to(device, dtype = torch.long)
            token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)
            targets = data['targets'].to(device, dtype = torch.float)
            outputs = model(ids, mask, token_type_ids)
            loss = loss_fn(outputs, targets)
            losses.append(loss.item())

            fin_targets.extend(targets.cpu().detach().numpy())
            fin_outputs.extend(torch.sigmoid(outputs).cpu().detach().numpy())
    print(f'Loss:  {np.mean(losses):.2f}')
    return fin_outputs, fin_targets, losses

In [43]:
dev_out, dev_tar, losses = evaluation()

Loss:  0.28


### Normal evaluation

In [44]:
outputs = np.array(dev_out) >= 0.5
targets = dev_tar
accuracy = metrics.accuracy_score(targets, outputs)
f1_score_micro = metrics.f1_score(targets, outputs, average='micro')
f1_score_macro = metrics.f1_score(targets, outputs, average='macro')

print(f"F1 Score (Micro) = {f1_score_micro}")
print(f"F1 Score (Macro) = {f1_score_macro}")

F1 Score (Micro) = 0.7950324597862803
F1 Score (Macro) = 0.7866978081048838


In [45]:
print(classification_report(targets, outputs, target_names=icd_classes_50, digits=4))

              precision    recall  f1-score   support

      250.00     0.7318    0.7620    0.7466      3004
       272.4     0.7554    0.7860    0.7704      3005
       401.9     0.8490    0.7569    0.8003      5721
      414.01     0.8528    0.8025    0.8269      3291
      427.31     0.8095    0.8665    0.8370      4653
       428.0     0.7011    0.9124    0.7929      4565
      518.81     0.7732    0.9028    0.8330      5494
      530.81     0.8607    0.6921    0.7672      2017
       584.9     0.7447    0.8291    0.7846      4412
       599.0     0.7427    0.6766    0.7081      2696

   micro avg     0.7763    0.8147    0.7950     38858
   macro avg     0.7821    0.7987    0.7867     38858
weighted avg     0.7816    0.8147    0.7943     38858
 samples avg     0.7576    0.8039    0.7585     38858



In [46]:
ruc_auc_score_micro = metrics.roc_auc_score(targets, outputs, average='micro')
ruc_auc_score_macro = metrics.roc_auc_score(targets, outputs, average='macro')

print(f"RUC AUC Score (Micro) = {ruc_auc_score_micro}")
print(f"RUC AUC Score (Macro) = {ruc_auc_score_macro}")

RUC AUC Score (Micro) = 0.8616141392695774
RUC AUC Score (Macro) = 0.8512430936908292


In [47]:
dev_df['prediction'] = dev_out
dev_df['tar'] = dev_tar

In [48]:
dev_df

,id,text,labels,prediction,tar
0,155705,NoteNursing/other nursing progress note please...,"[0, 0, 0, 0, 0, 1, 0, 0, 0, 1]","[0.6555914, 0.122174464, 0.4977138, 0.5236155,...","[0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, ..."
1,127366,NoteNursing/other condition update see carevue...,"[0, 0, 1, 0, 0, 0, 0, 0, 0, 0]","[0.09731535, 0.04754988, 0.5864375, 0.4050456,...","[0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
2,193135,NoteNursing/other neuro pt currently sedated o...,"[1, 0, 0, 0, 0, 1, 0, 0, 0, 0]","[0.14136018, 0.0046230494, 0.1579529, 0.099965...","[1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, ..."
3,146684,NoteNursing m w systolic chf w ef pulm htn osa...,"[0, 0, 0, 0, 1, 1, 0, 0, 1, 0]","[0.0015336872, 0.0037578498, 0.004839993, 0.00...","[0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, ..."
4,174278,NoteNursing vessel in systolic heart failure o...,"[0, 0, 0, 0, 0, 1, 0, 0, 1, 0]","[9.042427e-05, 0.001242115, 0.8252912, 0.00166...","[0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, ..."
...,...,...,...,...,...
13855,167558,NoteDischarge summary admission date discharge...,"[0, 0, 1, 0, 0, 0, 0, 0, 1, 0]","[0.073265575, 0.27431533, 0.669791, 0.0757443,...","[0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, ..."
13856,109739,NoteDischarge summary admission date discharge...,"[0, 0, 0, 0, 0, 0, 0, 0, 1, 1]","[0.76905257, 0.0769458, 0.31005344, 0.02509917...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, ..."
13857,152158,NoteDischarge summary admission date discharge...,"[0, 0, 1, 0, 0, 0, 1, 0, 0, 0]","[0.007976486, 0.01416777, 0.64938194, 0.001986...","[0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, ..."
13858,121027,NoteDischarge summary admission date discharge...,"[0, 0, 0, 0, 0, 0, 1, 0, 0, 0]","[0.11739641, 0.07751045, 0.16264074, 0.0604091...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, ..."


### Avg outputs

In [49]:
out_mean_dict = dev_df.groupby('id').prediction.apply(np.mean).to_dict()
dev_df['out_mean'] = dev_df['id'].map(out_mean_dict)
dev_df

,id,text,labels,prediction,tar,out_mean
0,155705,NoteNursing/other nursing progress note please...,"[0, 0, 0, 0, 0, 1, 0, 0, 0, 1]","[0.6555914, 0.122174464, 0.4977138, 0.5236155,...","[0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, ...","[0.23409514, 0.05197384, 0.21010317, 0.1615663..."
1,127366,NoteNursing/other condition update see carevue...,"[0, 0, 1, 0, 0, 0, 0, 0, 0, 0]","[0.09731535, 0.04754988, 0.5864375, 0.4050456,...","[0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.17044733, 0.10910279, 0.38751104, 0.1690149..."
2,193135,NoteNursing/other neuro pt currently sedated o...,"[1, 0, 0, 0, 0, 1, 0, 0, 0, 0]","[0.14136018, 0.0046230494, 0.1579529, 0.099965...","[1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, ...","[0.25141436, 0.023215936, 0.2776571, 0.1126098..."
3,146684,NoteNursing m w systolic chf w ef pulm htn osa...,"[0, 0, 0, 0, 1, 1, 0, 0, 1, 0]","[0.0015336872, 0.0037578498, 0.004839993, 0.00...","[0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, ...","[0.02149383, 0.011772488, 0.018777562, 0.00400..."
4,174278,NoteNursing vessel in systolic heart failure o...,"[0, 0, 0, 0, 0, 1, 0, 0, 1, 0]","[9.042427e-05, 0.001242115, 0.8252912, 0.00166...","[0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, ...","[0.06006429, 0.037939493, 0.41639563, 0.030593..."
...,...,...,...,...,...,...
13855,167558,NoteDischarge summary admission date discharge...,"[0, 0, 1, 0, 0, 0, 0, 0, 1, 0]","[0.073265575, 0.27431533, 0.669791, 0.0757443,...","[0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, ...","[0.17193964, 0.17212975, 0.6378448, 0.0564181,..."
13856,109739,NoteDischarge summary admission date discharge...,"[0, 0, 0, 0, 0, 0, 0, 0, 1, 1]","[0.76905257, 0.0769458, 0.31005344, 0.02509917...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, ...","[0.4144214, 0.13076, 0.25236556, 0.09724519, 0..."
13857,152158,NoteDischarge summary admission date discharge...,"[0, 0, 1, 0, 0, 0, 1, 0, 0, 0]","[0.007976486, 0.01416777, 0.64938194, 0.001986...","[0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, ...","[0.30443373, 0.15607329, 0.792141, 0.02080129,..."
13858,121027,NoteDischarge summary admission date discharge...,"[0, 0, 0, 0, 0, 0, 1, 0, 0, 0]","[0.11739641, 0.07751045, 0.16264074, 0.0604091...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, ...","[0.11739641, 0.07751045, 0.16264074, 0.0604091..."


In [50]:
df_mean = dev_df.drop_duplicates('id')

In [51]:
loss_mean = [nn.BCELoss()(torch.tensor(df_mean['out_mean'][i]), torch.tensor(df_mean['tar'][i])) for i in df_mean.index]
np.mean(loss_mean)

0.37415347

In [52]:
out_mean = np.vstack([df_mean['out_mean'][i]>=0.5 for i in df_mean.index])
targets = np.vstack([df_mean['tar'][i] for i in df_mean.index])
#targets = dev_tar
accuracy = metrics.accuracy_score(targets, out_mean)
f1_score_micro = metrics.f1_score(targets, out_mean, average='micro')
f1_score_macro = metrics.f1_score(targets, out_mean, average='macro')

print(f"F1 Score (Micro) = {f1_score_micro}")
print(f"F1 Score (Macro) = {f1_score_macro}")

F1 Score (Micro) = 0.6978142520516779
F1 Score (Macro) = 0.6820483842308801


In [53]:
print(classification_report(targets, out_mean, target_names=icd_classes_50, digits=4))

              precision    recall  f1-score   support

      250.00     0.6321    0.6515    0.6416      1010
       272.4     0.7150    0.6759    0.6949      1080
       401.9     0.8063    0.6735    0.7339      2181
      414.01     0.8259    0.7534    0.7880      1253
      427.31     0.7026    0.8013    0.7487      1409
       428.0     0.5937    0.8867    0.7112      1447
      518.81     0.6086    0.7737    0.6813      1043
      530.81     0.8083    0.4897    0.6099       680
       584.9     0.6090    0.7293    0.6638      1145
       599.0     0.6349    0.4808    0.5472       832

   micro avg     0.6852    0.7109    0.6978     12080
   macro avg     0.6936    0.6916    0.6820     12080
weighted avg     0.7006    0.7109    0.6963     12080
 samples avg     0.6616    0.7079    0.6550     12080



In [54]:
ruc_auc_score_micro = metrics.roc_auc_score(targets, out_mean, average='micro')
ruc_auc_score_macro = metrics.roc_auc_score(targets, out_mean, average='macro')

print(f"RUC AUC Score (Micro) = {ruc_auc_score_micro}")
print(f"RUC AUC Score (Macro) = {ruc_auc_score_macro}")

RUC AUC Score (Micro) = 0.7978240786421422
RUC AUC Score (Macro) = 0.7860991763241034


### Most freq 3 labels

In [55]:
out_sum_dict = dev_df.groupby('id').prediction.apply(np.sum).to_dict()
dev_df['out_sum'] = dev_df['id'].map(out_sum_dict)
dev_df

,id,text,labels,prediction,tar,out_mean,out_sum
0,155705,NoteNursing/other nursing progress note please...,"[0, 0, 0, 0, 0, 1, 0, 0, 0, 1]","[0.6555914, 0.122174464, 0.4977138, 0.5236155,...","[0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, ...","[0.23409514, 0.05197384, 0.21010317, 0.1615663...","[2.8091416, 0.6236861, 2.521238, 1.9387964, 5...."
1,127366,NoteNursing/other condition update see carevue...,"[0, 0, 1, 0, 0, 0, 0, 0, 0, 0]","[0.09731535, 0.04754988, 0.5864375, 0.4050456,...","[0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.17044733, 0.10910279, 0.38751104, 0.1690149...","[0.68178934, 0.43641117, 1.5500442, 0.67605984..."
2,193135,NoteNursing/other neuro pt currently sedated o...,"[1, 0, 0, 0, 0, 1, 0, 0, 0, 0]","[0.14136018, 0.0046230494, 0.1579529, 0.099965...","[1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, ...","[0.25141436, 0.023215936, 0.2776571, 0.1126098...","[1.0056574, 0.092863746, 1.1106284, 0.45043948..."
3,146684,NoteNursing m w systolic chf w ef pulm htn osa...,"[0, 0, 0, 0, 1, 1, 0, 0, 1, 0]","[0.0015336872, 0.0037578498, 0.004839993, 0.00...","[0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, ...","[0.02149383, 0.011772488, 0.018777562, 0.00400...","[0.7092964, 0.3884921, 0.61965954, 0.13211353,..."
4,174278,NoteNursing vessel in systolic heart failure o...,"[0, 0, 0, 0, 0, 1, 0, 0, 1, 0]","[9.042427e-05, 0.001242115, 0.8252912, 0.00166...","[0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, ...","[0.06006429, 0.037939493, 0.41639563, 0.030593...","[0.12012858, 0.075878985, 0.83279127, 0.061187..."
...,...,...,...,...,...,...,...
13855,167558,NoteDischarge summary admission date discharge...,"[0, 0, 1, 0, 0, 0, 0, 0, 1, 0]","[0.073265575, 0.27431533, 0.669791, 0.0757443,...","[0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, ...","[0.17193964, 0.17212975, 0.6378448, 0.0564181,...","[0.68775856, 0.688519, 2.5513792, 0.2256724, 0..."
13856,109739,NoteDischarge summary admission date discharge...,"[0, 0, 0, 0, 0, 0, 0, 0, 1, 1]","[0.76905257, 0.0769458, 0.31005344, 0.02509917...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, ...","[0.4144214, 0.13076, 0.25236556, 0.09724519, 0...","[1.2432642, 0.39227998, 0.75709665, 0.29173556..."
13857,152158,NoteDischarge summary admission date discharge...,"[0, 0, 1, 0, 0, 0, 1, 0, 0, 0]","[0.007976486, 0.01416777, 0.64938194, 0.001986...","[0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, ...","[0.30443373, 0.15607329, 0.792141, 0.02080129,...","[0.91330117, 0.46821985, 2.3764231, 0.06240387..."
13858,121027,NoteDischarge summary admission date discharge...,"[0, 0, 0, 0, 0, 0, 1, 0, 0, 0]","[0.11739641, 0.07751045, 0.16264074, 0.0604091...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, ...","[0.11739641, 0.07751045, 0.16264074, 0.0604091...","[0.11739641, 0.07751045, 0.16264074, 0.0604091..."


In [56]:
def freq_3(df, column): # column: out_sum
    df['freq_3'] = df[column]
    for idx in df.index:
      sorted = np.sort(df[column][idx])
      thres = sorted[-3] # position 3
      df['freq_3'][idx] = df[column][idx]>= thres


In [57]:
freq_3(dev_df, 'out_sum')

In [58]:
dev_df

,id,text,labels,prediction,tar,out_mean,out_sum,freq_3
0,155705,NoteNursing/other nursing progress note please...,"[0, 0, 0, 0, 0, 1, 0, 0, 0, 1]","[0.6555914, 0.122174464, 0.4977138, 0.5236155,...","[0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, ...","[0.23409514, 0.05197384, 0.21010317, 0.1615663...","[2.8091416, 0.6236861, 2.521238, 1.9387964, 5....","[False, False, False, False, False, True, True..."
1,127366,NoteNursing/other condition update see carevue...,"[0, 0, 1, 0, 0, 0, 0, 0, 0, 0]","[0.09731535, 0.04754988, 0.5864375, 0.4050456,...","[0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.17044733, 0.10910279, 0.38751104, 0.1690149...","[0.68178934, 0.43641117, 1.5500442, 0.67605984...","[False, False, True, False, False, True, True,..."
2,193135,NoteNursing/other neuro pt currently sedated o...,"[1, 0, 0, 0, 0, 1, 0, 0, 0, 0]","[0.14136018, 0.0046230494, 0.1579529, 0.099965...","[1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, ...","[0.25141436, 0.023215936, 0.2776571, 0.1126098...","[1.0056574, 0.092863746, 1.1106284, 0.45043948...","[False, False, False, False, False, True, True..."
3,146684,NoteNursing m w systolic chf w ef pulm htn osa...,"[0, 0, 0, 0, 1, 1, 0, 0, 1, 0]","[0.0015336872, 0.0037578498, 0.004839993, 0.00...","[0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, ...","[0.02149383, 0.011772488, 0.018777562, 0.00400...","[0.7092964, 0.3884921, 0.61965954, 0.13211353,...","[False, False, False, False, True, True, False..."
4,174278,NoteNursing vessel in systolic heart failure o...,"[0, 0, 0, 0, 0, 1, 0, 0, 1, 0]","[9.042427e-05, 0.001242115, 0.8252912, 0.00166...","[0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, ...","[0.06006429, 0.037939493, 0.41639563, 0.030593...","[0.12012858, 0.075878985, 0.83279127, 0.061187...","[False, False, False, False, True, True, False..."
...,...,...,...,...,...,...,...,...
13855,167558,NoteDischarge summary admission date discharge...,"[0, 0, 1, 0, 0, 0, 0, 0, 1, 0]","[0.073265575, 0.27431533, 0.669791, 0.0757443,...","[0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, ...","[0.17193964, 0.17212975, 0.6378448, 0.0564181,...","[0.68775856, 0.688519, 2.5513792, 0.2256724, 0...","[False, False, True, False, False, False, True..."
13856,109739,NoteDischarge summary admission date discharge...,"[0, 0, 0, 0, 0, 0, 0, 0, 1, 1]","[0.76905257, 0.0769458, 0.31005344, 0.02509917...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, ...","[0.4144214, 0.13076, 0.25236556, 0.09724519, 0...","[1.2432642, 0.39227998, 0.75709665, 0.29173556...","[True, False, False, False, False, True, False..."
13857,152158,NoteDischarge summary admission date discharge...,"[0, 0, 1, 0, 0, 0, 1, 0, 0, 0]","[0.007976486, 0.01416777, 0.64938194, 0.001986...","[0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, ...","[0.30443373, 0.15607329, 0.792141, 0.02080129,...","[0.91330117, 0.46821985, 2.3764231, 0.06240387...","[True, False, True, False, False, False, True,..."
13858,121027,NoteDischarge summary admission date discharge...,"[0, 0, 0, 0, 0, 0, 1, 0, 0, 0]","[0.11739641, 0.07751045, 0.16264074, 0.0604091...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, ...","[0.11739641, 0.07751045, 0.16264074, 0.0604091...","[0.11739641, 0.07751045, 0.16264074, 0.0604091...","[False, False, False, False, False, True, True..."


In [59]:
df_freq_3 = dev_df.drop_duplicates('id')

In [60]:
out_freq_3 = np.vstack([df_freq_3['freq_3'][i] for i in df_freq_3.index])
targets = np.vstack([df_freq_3['tar'][i] for i in df_freq_3.index])
#targets = dev_tar
accuracy = metrics.accuracy_score(targets, out_freq_3)
f1_score_micro = metrics.f1_score(targets, out_freq_3, average='micro')
f1_score_macro = metrics.f1_score(targets, out_freq_3, average='macro')

print(f"F1 Score (Micro) = {f1_score_micro}")
print(f"F1 Score (Macro) = {f1_score_macro}")

F1 Score (Micro) = 0.6241866553728872
F1 Score (Macro) = 0.6050554321285563


In [61]:
print(classification_report(targets, out_freq_3, target_names=icd_classes_50, digits=4))

              precision    recall  f1-score   support

      250.00     0.5403    0.5713    0.5553      1010
       272.4     0.6068    0.5472    0.5755      1080
       401.9     0.7361    0.6534    0.6923      2181
      414.01     0.7945    0.6664    0.7248      1253
      427.31     0.6656    0.7559    0.7079      1409
       428.0     0.5142    0.8874    0.6511      1447
      518.81     0.4922    0.7833    0.6045      1043
      530.81     0.6157    0.4382    0.5120       680
       584.9     0.4837    0.7109    0.5757      1145
       599.0     0.4255    0.4808    0.4515       832

   micro avg     0.5835    0.6710    0.6242     12080
   macro avg     0.5874    0.6495    0.6051     12080
weighted avg     0.6063    0.6710    0.6253     12080
 samples avg     0.5835    0.7264    0.6037     12080



In [62]:
ruc_auc_score_micro = metrics.roc_auc_score(targets, out_freq_3, average='micro')
ruc_auc_score_macro = metrics.roc_auc_score(targets, out_freq_3, average='macro')

print(f"RUC AUC Score (Micro) = {ruc_auc_score_micro}")
print(f"RUC AUC Score (Macro) = {ruc_auc_score_macro}")

RUC AUC Score (Micro) = 0.7509821319743922
RUC AUC Score (Macro) = 0.7379146266179182


### Predicted percentage

In [63]:
note_count_dict = dev_df.groupby('id').size().to_dict()
dev_df['note_count'] = dev_df['id'].map(note_count_dict)

In [64]:
dev_df['out_bool'] = [(dev_df['prediction'][i]>=0.5).astype(int) for i in dev_df.index]

In [65]:

out_freq_dict = dev_df.groupby('id').out_bool.apply(np.sum).to_dict()
dev_df['num_pred'] = dev_df['id'].map(out_freq_dict)
dev_df['num_pred'] = [(dev_df['num_pred'][i]>=0.6*dev_df['note_count'][i]).astype(int) for i in dev_df.index]

In [66]:
dev_df

,id,text,labels,prediction,tar,out_mean,out_sum,freq_3,note_count,out_bool,num_pred
0,155705,NoteNursing/other nursing progress note please...,"[0, 0, 0, 0, 0, 1, 0, 0, 0, 1]","[0.6555914, 0.122174464, 0.4977138, 0.5236155,...","[0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, ...","[0.23409514, 0.05197384, 0.21010317, 0.1615663...","[2.8091416, 0.6236861, 2.521238, 1.9387964, 5....","[False, False, False, False, False, True, True...",12,"[1, 0, 0, 1, 1, 1, 0, 0, 1, 0]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
1,127366,NoteNursing/other condition update see carevue...,"[0, 0, 1, 0, 0, 0, 0, 0, 0, 0]","[0.09731535, 0.04754988, 0.5864375, 0.4050456,...","[0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.17044733, 0.10910279, 0.38751104, 0.1690149...","[0.68178934, 0.43641117, 1.5500442, 0.67605984...","[False, False, True, False, False, True, True,...",4,"[0, 0, 1, 0, 0, 1, 1, 0, 0, 0]","[0, 0, 0, 0, 0, 1, 0, 0, 0, 0]"
2,193135,NoteNursing/other neuro pt currently sedated o...,"[1, 0, 0, 0, 0, 1, 0, 0, 0, 0]","[0.14136018, 0.0046230494, 0.1579529, 0.099965...","[1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, ...","[0.25141436, 0.023215936, 0.2776571, 0.1126098...","[1.0056574, 0.092863746, 1.1106284, 0.45043948...","[False, False, False, False, False, True, True...",4,"[0, 0, 0, 0, 0, 1, 1, 0, 0, 0]","[0, 0, 0, 0, 0, 0, 1, 0, 0, 0]"
3,146684,NoteNursing m w systolic chf w ef pulm htn osa...,"[0, 0, 0, 0, 1, 1, 0, 0, 1, 0]","[0.0015336872, 0.0037578498, 0.004839993, 0.00...","[0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, ...","[0.02149383, 0.011772488, 0.018777562, 0.00400...","[0.7092964, 0.3884921, 0.61965954, 0.13211353,...","[False, False, False, False, True, True, False...",33,"[0, 0, 0, 0, 1, 1, 0, 0, 1, 0]","[0, 0, 0, 0, 1, 1, 0, 0, 1, 0]"
4,174278,NoteNursing vessel in systolic heart failure o...,"[0, 0, 0, 0, 0, 1, 0, 0, 1, 0]","[9.042427e-05, 0.001242115, 0.8252912, 0.00166...","[0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, ...","[0.06006429, 0.037939493, 0.41639563, 0.030593...","[0.12012858, 0.075878985, 0.83279127, 0.061187...","[False, False, False, False, True, True, False...",2,"[0, 0, 1, 0, 1, 1, 1, 0, 0, 0]","[0, 0, 0, 0, 0, 1, 0, 0, 0, 0]"
...,...,...,...,...,...,...,...,...,...,...,...
13855,167558,NoteDischarge summary admission date discharge...,"[0, 0, 1, 0, 0, 0, 0, 0, 1, 0]","[0.073265575, 0.27431533, 0.669791, 0.0757443,...","[0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, ...","[0.17193964, 0.17212975, 0.6378448, 0.0564181,...","[0.68775856, 0.688519, 2.5513792, 0.2256724, 0...","[False, False, True, False, False, False, True...",4,"[0, 0, 1, 0, 0, 0, 0, 0, 0, 0]","[0, 0, 1, 0, 0, 0, 0, 0, 0, 0]"
13856,109739,NoteDischarge summary admission date discharge...,"[0, 0, 0, 0, 0, 0, 0, 0, 1, 1]","[0.76905257, 0.0769458, 0.31005344, 0.02509917...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, ...","[0.4144214, 0.13076, 0.25236556, 0.09724519, 0...","[1.2432642, 0.39227998, 0.75709665, 0.29173556...","[True, False, False, False, False, True, False...",3,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
13857,152158,NoteDischarge summary admission date discharge...,"[0, 0, 1, 0, 0, 0, 1, 0, 0, 0]","[0.007976486, 0.01416777, 0.64938194, 0.001986...","[0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, ...","[0.30443373, 0.15607329, 0.792141, 0.02080129,...","[0.91330117, 0.46821985, 2.3764231, 0.06240387...","[True, False, True, False, False, False, True,...",3,"[0, 0, 1, 0, 0, 0, 1, 0, 0, 0]","[0, 0, 1, 0, 0, 0, 1, 0, 0, 0]"
13858,121027,NoteDischarge summary admission date discharge...,"[0, 0, 0, 0, 0, 0, 1, 0, 0, 0]","[0.11739641, 0.07751045, 0.16264074, 0.0604091...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, ...","[0.11739641, 0.07751045, 0.16264074, 0.0604091...","[0.11739641, 0.07751045, 0.16264074, 0.0604091...","[False, False, False, False, False, True, True...",1,"[0, 0, 0, 0, 0, 1, 1, 0, 1, 0]","[0, 0, 0, 0, 0, 1, 1, 0, 1, 0]"


In [67]:
df_freq = dev_df.drop_duplicates('id')

In [68]:
out_freq = np.vstack([df_freq['num_pred'][i] for i in df_freq.index])
targets = np.vstack([df_freq['tar'][i] for i in df_freq.index])

#targets = dev_tar
accuracy = metrics.accuracy_score(targets, out_freq)
f1_score_micro = metrics.f1_score(targets, out_freq, average='micro')
f1_score_macro = metrics.f1_score(targets, out_freq, average='macro')

print(f"F1 Score (Micro) = {f1_score_micro}")
print(f"F1 Score (Macro) = {f1_score_macro}")

F1 Score (Micro) = 0.6786604164027197
F1 Score (Macro) = 0.660390043732423


In [69]:
print(classification_report(targets, out_freq, target_names=icd_classes_50, digits=4))

              precision    recall  f1-score   support

      250.00     0.6387    0.6040    0.6209      1010
       272.4     0.7152    0.6324    0.6713      1080
       401.9     0.8096    0.6295    0.7083      2181
      414.01     0.8333    0.7103    0.7669      1253
      427.31     0.7067    0.7594    0.7321      1409
       428.0     0.6016    0.8452    0.7029      1447
      518.81     0.6320    0.7344    0.6794      1043
      530.81     0.8037    0.4456    0.5733       680
       584.9     0.6224    0.6707    0.6456      1145
       599.0     0.6288    0.4195    0.5032       832

   micro avg     0.6927    0.6651    0.6787     12080
   macro avg     0.6992    0.6451    0.6604     12080
weighted avg     0.7065    0.6651    0.6757     12080
 samples avg     0.6486    0.6620    0.6247     12080



In [70]:
ruc_auc_score_micro = metrics.roc_auc_score(targets, out_freq, average='micro')
ruc_auc_score_macro = metrics.roc_auc_score(targets, out_freq, average='macro')

print(f"RUC AUC Score (Micro) = {ruc_auc_score_micro}")
print(f"RUC AUC Score (Macro) = {ruc_auc_score_macro}")

RUC AUC Score (Micro) = 0.7805149064663853
RUC AUC Score (Macro) = 0.7685736183162598


### Exponential moving average

In [71]:
def ewma(sub_df, window=3):
#    print(sub_df)
    alpha = 2 / (window + 1)
#    print(sub_df['ewma'])
    sub_df['ewma'] = sub_df['out_bool']
    for r in sub_df.index:
        if r == sub_df.index[0]:
            sub_df['ewma'][r] = sub_df['prediction'][r]
        else:
            sub_df['ewma'][r] = alpha*sub_df['prediction'][r] + (1-alpha)*sub_df['prediction'][sub_df.index[sub_df.index.get_loc(r)-1]]
 #   print(type(sub_df['ewma']))
    return sub_df['ewma']

In [72]:

out_ewma_dict = dev_df.groupby('id', group_keys=False).apply(ewma).to_dict()
dev_df['out_ewma'] = pd.Series(dev_df.index, index=dev_df.index).map(out_ewma_dict)



In [73]:
dev_df

,id,text,labels,prediction,tar,out_mean,out_sum,freq_3,note_count,out_bool,num_pred,out_ewma
0,155705,NoteNursing/other nursing progress note please...,"[0, 0, 0, 0, 0, 1, 0, 0, 0, 1]","[0.6555914, 0.122174464, 0.4977138, 0.5236155,...","[0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, ...","[0.23409514, 0.05197384, 0.21010317, 0.1615663...","[2.8091416, 0.6236861, 2.521238, 1.9387964, 5....","[False, False, False, False, False, True, True...",12,"[1, 0, 0, 1, 1, 1, 0, 0, 1, 0]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[0.6555914, 0.122174464, 0.4977138, 0.5236155,..."
1,127366,NoteNursing/other condition update see carevue...,"[0, 0, 1, 0, 0, 0, 0, 0, 0, 0]","[0.09731535, 0.04754988, 0.5864375, 0.4050456,...","[0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.17044733, 0.10910279, 0.38751104, 0.1690149...","[0.68178934, 0.43641117, 1.5500442, 0.67605984...","[False, False, True, False, False, True, True,...",4,"[0, 0, 1, 0, 0, 1, 1, 0, 0, 0]","[0, 0, 0, 0, 0, 1, 0, 0, 0, 0]","[0.09731535, 0.04754988, 0.5864375, 0.4050456,..."
2,193135,NoteNursing/other neuro pt currently sedated o...,"[1, 0, 0, 0, 0, 1, 0, 0, 0, 0]","[0.14136018, 0.0046230494, 0.1579529, 0.099965...","[1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, ...","[0.25141436, 0.023215936, 0.2776571, 0.1126098...","[1.0056574, 0.092863746, 1.1106284, 0.45043948...","[False, False, False, False, False, True, True...",4,"[0, 0, 0, 0, 0, 1, 1, 0, 0, 0]","[0, 0, 0, 0, 0, 0, 1, 0, 0, 0]","[0.14136018, 0.0046230494, 0.1579529, 0.099965..."
3,146684,NoteNursing m w systolic chf w ef pulm htn osa...,"[0, 0, 0, 0, 1, 1, 0, 0, 1, 0]","[0.0015336872, 0.0037578498, 0.004839993, 0.00...","[0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, ...","[0.02149383, 0.011772488, 0.018777562, 0.00400...","[0.7092964, 0.3884921, 0.61965954, 0.13211353,...","[False, False, False, False, True, True, False...",33,"[0, 0, 0, 0, 1, 1, 0, 0, 1, 0]","[0, 0, 0, 0, 1, 1, 0, 0, 1, 0]","[0.0015336872, 0.0037578498, 0.004839993, 0.00..."
4,174278,NoteNursing vessel in systolic heart failure o...,"[0, 0, 0, 0, 0, 1, 0, 0, 1, 0]","[9.042427e-05, 0.001242115, 0.8252912, 0.00166...","[0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, ...","[0.06006429, 0.037939493, 0.41639563, 0.030593...","[0.12012858, 0.075878985, 0.83279127, 0.061187...","[False, False, False, False, True, True, False...",2,"[0, 0, 1, 0, 1, 1, 1, 0, 0, 0]","[0, 0, 0, 0, 0, 1, 0, 0, 0, 0]","[9.042427e-05, 0.001242115, 0.8252912, 0.00166..."
...,...,...,...,...,...,...,...,...,...,...,...,...
13855,167558,NoteDischarge summary admission date discharge...,"[0, 0, 1, 0, 0, 0, 0, 0, 1, 0]","[0.073265575, 0.27431533, 0.669791, 0.0757443,...","[0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, ...","[0.17193964, 0.17212975, 0.6378448, 0.0564181,...","[0.68775856, 0.688519, 2.5513792, 0.2256724, 0...","[False, False, True, False, False, False, True...",4,"[0, 0, 1, 0, 0, 0, 0, 0, 0, 0]","[0, 0, 1, 0, 0, 0, 0, 0, 0, 0]","[0.22744724, 0.2504483, 0.63824445, 0.05259072..."
13856,109739,NoteDischarge summary admission date discharge...,"[0, 0, 0, 0, 0, 0, 0, 0, 1, 1]","[0.76905257, 0.0769458, 0.31005344, 0.02509917...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, ...","[0.4144214, 0.13076, 0.25236556, 0.09724519, 0...","[1.2432642, 0.39227998, 0.75709665, 0.29173556...","[True, False, False, False, False, True, False...",3,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[0.4728411, 0.11834861, 0.30081028, 0.05009076..."
13857,152158,NoteDischarge summary admission date discharge...,"[0, 0, 1, 0, 0, 0, 1, 0, 0, 0]","[0.007976486, 0.01416777, 0.64938194, 0.001986...","[0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, ...","[0.30443373, 0.15607329, 0.792141, 0.02080129,...","[0.91330117, 0.46821985, 2.3764231, 0.06240387...","[True, False, True, False, False, False, True,...",3,"[0, 0, 1, 0, 0, 0, 1, 0, 0, 0]","[0, 0, 1, 0, 0, 0, 1, 0, 0, 0]","[0.3246124, 0.13611805, 0.7646976, 0.003632738..."
13858,121027,NoteDischarge summary admission date discharge...,"[

In [74]:
df_ewma = dev_df.drop_duplicates('id')

In [75]:
out_ewma = np.vstack([df_ewma['out_ewma'][i]>0.5 for i in df_ewma.index])
targets = np.vstack([df_ewma['tar'][i] for i in df_ewma.index])

#targets = dev_tar
accuracy = metrics.accuracy_score(targets, out_ewma)
f1_score_micro = metrics.f1_score(targets, out_ewma, average='micro')
f1_score_macro = metrics.f1_score(targets, out_ewma, average='macro')

print(f"F1 Score (Micro) = {f1_score_micro}")
print(f"F1 Score (Macro) = {f1_score_macro}")

F1 Score (Micro) = 0.6416559897501601
F1 Score (Macro) = 0.6224568582211765


In [76]:
print(classification_report(targets, out_ewma, target_names=icd_classes_50, digits=4))

              precision    recall  f1-score   support

      250.00     0.5337    0.5960    0.5631      1010
       272.4     0.6319    0.6231    0.6275      1080
       401.9     0.7600    0.6272    0.6873      2181
      414.01     0.7734    0.7111    0.7410      1253
      427.31     0.6513    0.7608    0.7018      1409
       428.0     0.5553    0.8542    0.6730      1447
      518.81     0.5409    0.7220    0.6185      1043
      530.81     0.7228    0.4294    0.5387       680
       584.9     0.5391    0.6803    0.6015      1145
       599.0     0.5439    0.4171    0.4721       832

   micro avg     0.6214    0.6633    0.6417     12080
   macro avg     0.6252    0.6421    0.6225     12080
weighted avg     0.6370    0.6633    0.6399     12080
 samples avg     0.5979    0.6622    0.5961     12080



In [77]:
ruc_auc_score_micro = metrics.roc_auc_score(targets, out_ewma, average='micro')
ruc_auc_score_macro = metrics.roc_auc_score(targets, out_ewma, average='macro')

print(f"RUC AUC Score (Micro) = {ruc_auc_score_micro}")
print(f"RUC AUC Score (Macro) = {ruc_auc_score_macro}")

RUC AUC Score (Micro) = 0.7603375853449494
RUC AUC Score (Macro) = 0.7474444425918849


### Most frequent prediction

In [78]:

most_freq_dict = dev_df.groupby('id')['out_bool'].apply(lambda x: x.value_counts().index[0]).to_dict()
dev_df['most_freq'] = dev_df['id'].map(most_freq_dict)

In [79]:
dev_df

,id,text,labels,prediction,tar,out_mean,out_sum,freq_3,note_count,out_bool,num_pred,out_ewma,most_freq
0,155705,NoteNursing/other nursing progress note please...,"[0, 0, 0, 0, 0, 1, 0, 0, 0, 1]","[0.6555914, 0.122174464, 0.4977138, 0.5236155,...","[0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, ...","[0.23409514, 0.05197384, 0.21010317, 0.1615663...","[2.8091416, 0.6236861, 2.521238, 1.9387964, 5....","[False, False, False, False, False, True, True...",12,"[1, 0, 0, 1, 1, 1, 0, 0, 1, 0]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[0.6555914, 0.122174464, 0.4977138, 0.5236155,...","[0, 0, 0, 0, 1, 0, 0, 0, 0, 1]"
1,127366,NoteNursing/other condition update see carevue...,"[0, 0, 1, 0, 0, 0, 0, 0, 0, 0]","[0.09731535, 0.04754988, 0.5864375, 0.4050456,...","[0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.17044733, 0.10910279, 0.38751104, 0.1690149...","[0.68178934, 0.43641117, 1.5500442, 0.67605984...","[False, False, True, False, False, True, True,...",4,"[0, 0, 1, 0, 0, 1, 1, 0, 0, 0]","[0, 0, 0, 0, 0, 1, 0, 0, 0, 0]","[0.09731535, 0.04754988, 0.5864375, 0.4050456,...","[0, 0, 0, 0, 0, 1, 0, 0, 0, 0]"
2,193135,NoteNursing/other neuro pt currently sedated o...,"[1, 0, 0, 0, 0, 1, 0, 0, 0, 0]","[0.14136018, 0.0046230494, 0.1579529, 0.099965...","[1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, ...","[0.25141436, 0.023215936, 0.2776571, 0.1126098...","[1.0056574, 0.092863746, 1.1106284, 0.45043948...","[False, False, False, False, False, True, True...",4,"[0, 0, 0, 0, 0, 1, 1, 0, 0, 0]","[0, 0, 0, 0, 0, 0, 1, 0, 0, 0]","[0.14136018, 0.0046230494, 0.1579529, 0.099965...","[0, 0, 1, 0, 0, 0, 1, 0, 0, 0]"
3,146684,NoteNursing m w systolic chf w ef pulm htn osa...,"[0, 0, 0, 0, 1, 1, 0, 0, 1, 0]","[0.0015336872, 0.0037578498, 0.004839993, 0.00...","[0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, ...","[0.02149383, 0.011772488, 0.018777562, 0.00400...","[0.7092964, 0.3884921, 0.61965954, 0.13211353,...","[False, False, False, False, True, True, False...",33,"[0, 0, 0, 0, 1, 1, 0, 0, 1, 0]","[0, 0, 0, 0, 1, 1, 0, 0, 1, 0]","[0.0015336872, 0.0037578498, 0.004839993, 0.00...","[0, 0, 0, 0, 1, 1, 0, 0, 1, 0]"
4,174278,NoteNursing vessel in systolic heart failure o...,"[0, 0, 0, 0, 0, 1, 0, 0, 1, 0]","[9.042427e-05, 0.001242115, 0.8252912, 0.00166...","[0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, ...","[0.06006429, 0.037939493, 0.41639563, 0.030593...","[0.12012858, 0.075878985, 0.83279127, 0.061187...","[False, False, False, False, True, True, False...",2,"[0, 0, 1, 0, 1, 1, 1, 0, 0, 0]","[0, 0, 0, 0, 0, 1, 0, 0, 0, 0]","[9.042427e-05, 0.001242115, 0.8252912, 0.00166...","[0, 0, 1, 0, 1, 1, 1, 0, 0, 0]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...
13855,167558,NoteDischarge summary admission date discharge...,"[0, 0, 1, 0, 0, 0, 0, 0, 1, 0]","[0.073265575, 0.27431533, 0.669791, 0.0757443,...","[0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, ...","[0.17193964, 0.17212975, 0.6378448, 0.0564181,...","[0.68775856, 0.688519, 2.5513792, 0.2256724, 0...","[False, False, True, False, False, False, True...",4,"[0, 0, 1, 0, 0, 0, 0, 0, 0, 0]","[0, 0, 1, 0, 0, 0, 0, 0, 0, 0]","[0.22744724, 0.2504483, 0.63824445, 0.05259072...","[0, 0, 1, 0, 0, 0, 0, 0, 1, 0]"
13856,109739,NoteDischarge summary admission date discharge...,"[0, 0, 0, 0, 0, 0, 0, 0, 1, 1]","[0.76905257, 0.0769458, 0.31005344, 0.02509917...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, ...","[0.4144214, 0.13076, 0.25236556, 0.09724519, 0...","[1.2432642, 0.39227998, 0.75709665, 0.29173556...","[True, False, False, False, False, True, False...",3,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[0.4728411, 0.11834861, 0.30081028, 0.05009076...","[0, 0, 0, 0, 0, 1, 0, 0, 0, 0]"
13857,152158,NoteDischarge summary admission date discharge...,"[0, 0, 1, 0, 0, 0, 1, 0, 0, 0]","[0.007976486, 0.01416777, 0.64938194, 0.001986...","[0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, ...","[0.30443373, 0.15607329, 0.792141, 0.02080129,...","[0.91330117, 0.46821985, 2.3764231, 0.06240

In [80]:
df_most_freq = dev_df.drop_duplicates('id')

In [81]:
out_most_freq = np.vstack([df_most_freq['most_freq'][i] for i in df_most_freq.index])
targets = np.vstack([df_most_freq['tar'][i] for i in df_most_freq.index])
#targets = dev_tar
accuracy = metrics.accuracy_score(targets, out_most_freq)
f1_score_micro = metrics.f1_score(targets, out_most_freq, average='micro')
f1_score_macro = metrics.f1_score(targets, out_most_freq, average='macro')

print(f"F1 Score (Micro) = {f1_score_micro}")
print(f"F1 Score (Macro) = {f1_score_macro}")

F1 Score (Micro) = 0.6551765505658416
F1 Score (Macro) = 0.6371652209017435


In [82]:
print(classification_report(targets, out_most_freq, target_names=icd_classes_50, digits=4))

              precision    recall  f1-score   support

      250.00     0.5529    0.6109    0.5804      1010
       272.4     0.6516    0.6407    0.6461      1080
       401.9     0.7690    0.6442    0.7011      2181
      414.01     0.7850    0.7199    0.7510      1253
      427.31     0.6550    0.7679    0.7070      1409
       428.0     0.5657    0.8604    0.6826      1447
      518.81     0.5533    0.7411    0.6336      1043
      530.81     0.7303    0.4500    0.5569       680
       584.9     0.5566    0.6996    0.6200      1145
       599.0     0.5549    0.4435    0.4930       832

   micro avg     0.6337    0.6781    0.6552     12080
   macro avg     0.6374    0.6578    0.6372     12080
weighted avg     0.6488    0.6781    0.6538     12080
 samples avg     0.6116    0.6763    0.6104     12080



In [83]:
ruc_auc_score_micro = metrics.roc_auc_score(targets, out_most_freq, average='micro')
ruc_auc_score_macro = metrics.roc_auc_score(targets, out_most_freq, average='macro')

print(f"RUC AUC Score (Micro) = {ruc_auc_score_micro}")
print(f"RUC AUC Score (Macro) = {ruc_auc_score_macro}")

RUC AUC Score (Micro) = 0.7699083720759257
RUC AUC Score (Macro) = 0.7575127457060906


### Testing

In [84]:

# Evaluate the model

def testing():
    model.eval()

    fin_targets=[]
    fin_outputs=[]
    losses=[]
    with torch.no_grad():
        for _, data in enumerate(testing_loader, 0):
            ids = data['ids'].to(device, dtype = torch.long)
            mask = data['mask'].to(device, dtype = torch.long)
            token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)
            targets = data['targets'].to(device, dtype = torch.float)
            outputs = model(ids, mask, token_type_ids)
            loss = loss_fn(outputs, targets)
            losses.append(loss.item())

            fin_targets.extend(targets.cpu().detach().numpy())
            fin_outputs.extend(torch.sigmoid(outputs).cpu().detach().numpy())
    print(f'Loss:  {np.mean(losses):.2f}')
    return fin_outputs, fin_targets, losses

In [85]:
test_out, targets, losses = testing()
outputs = np.array(test_out) >= 0.5
accuracy = metrics.accuracy_score(targets, outputs)
f1_score_micro = metrics.f1_score(targets, outputs, average='micro')
f1_score_macro = metrics.f1_score(targets, outputs, average='macro')
print(f"Accuracy Score = {accuracy}")
print(f"F1 Score (Micro) = {f1_score_micro}")
print(f"F1 Score (Macro) = {f1_score_macro}")

Loss:  0.28
Accuracy Score = 0.49396825396825395
F1 Score (Micro) = 0.7972825867838568
F1 Score (Macro) = 0.7895650396974786


In [86]:
print(classification_report(targets, outputs, target_names=icd_classes_50, digits=4))

              precision    recall  f1-score   support

      250.00     0.7230    0.7637    0.7428      3677
       272.4     0.7640    0.7655    0.7648      3714
       401.9     0.8549    0.7491    0.7985      7167
      414.01     0.8542    0.8148    0.8340      4120
      427.31     0.8158    0.8620    0.8383      5817
       428.0     0.7179    0.9158    0.8048      5773
      518.81     0.7726    0.8956    0.8296      6743
      530.81     0.8655    0.7071    0.7783      2567
       584.9     0.7562    0.8289    0.7909      5610
       599.0     0.7365    0.6922    0.7137      3421

   micro avg     0.7816    0.8136    0.7973     48609
   macro avg     0.7861    0.7995    0.7896     48609
weighted avg     0.7864    0.8136    0.7966     48609
 samples avg     0.7625    0.8020    0.7602     48609



In [87]:

ruc_auc_score_micro = metrics.roc_auc_score(targets, outputs, average='micro')
ruc_auc_score_macro = metrics.roc_auc_score(targets, outputs, average='macro')

print(f"RUC AUC Score (Micro) = {ruc_auc_score_micro}")
print(f"RUC AUC Score (Macro) = {ruc_auc_score_macro}")

RUC AUC Score (Micro) = 0.8624783249182079
RUC AUC Score (Macro) = 0.8531996612817976


In [88]:
test_df['prediction'] = test_out
test_df['tar'] = targets

In [89]:
out_mean_dict = test_df.groupby('id').prediction.apply(np.mean).to_dict()
test_df['out_mean'] = test_df['id'].map(out_mean_dict)
test_df

,id,text,labels,prediction,tar,out_mean
0,151838,NoteEcho patient test information indication c...,"[0, 0, 0, 1, 0, 1, 0, 0, 1, 0]","[0.61245716, 0.47937575, 0.4841983, 0.75877136...","[0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, ...","[0.61245716, 0.47937575, 0.4841983, 0.75877136..."
1,135559,NoteRadiology pm pelvis limited pelvis u s tra...,"[0, 0, 0, 0, 0, 0, 1, 0, 0, 0]","[0.04677391, 0.79962146, 0.41040155, 0.0317791...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, ...","[0.12019126, 0.4994751, 0.3433295, 0.07496161,..."
2,132213,NoteNursing/other pmicu nursing progress 7a 7p...,"[0, 0, 0, 0, 0, 0, 0, 1, 0, 0]","[0.090464346, 0.03696774, 0.27864078, 0.055774...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, ...","[0.090464346, 0.03696774, 0.27864078, 0.055774..."
3,183196,NoteRadiology pm ct chest w contrast clip clip...,"[1, 0, 0, 1, 0, 0, 0, 0, 0, 0]","[0.18781413, 0.24339023, 0.25823396, 0.2707603...","[1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.12332781, 0.13642125, 0.21342212, 0.3034975..."
4,159509,NoteEcho patient test information indication l...,"[1, 0, 0, 0, 1, 1, 0, 0, 0, 0]","[0.34028667, 0.030640895, 0.1411787, 0.2678510...","[1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, ...","[0.34028667, 0.030640895, 0.1411787, 0.2678510..."
...,...,...,...,...,...,...
17320,150956,NoteDischarge summary admission date discharge...,"[0, 0, 0, 0, 1, 1, 1, 0, 0, 0]","[0.053910743, 0.0143957045, 0.1383739, 0.00822...","[0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, ...","[0.07297033, 0.18675524, 0.027424058, 0.069857..."
17321,160172,NoteDischarge summary admission date discharge...,"[0, 0, 0, 0, 0, 0, 1, 0, 0, 0]","[0.005538515, 0.005508835, 0.094535194, 0.0009...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, ...","[0.26569664, 0.30781466, 0.19839652, 0.1076491..."
17322,161772,NoteDischarge summary admission date discharge...,"[0, 0, 0, 0, 0, 0, 1, 0, 0, 0]","[0.027512353, 0.14406385, 0.62541795, 0.082501...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, ...","[0.05998641, 0.05926599, 0.21342896, 0.0486895..."
17323,197451,NoteDischarge summary admission date discharge...,"[0, 0, 1, 1, 0, 0, 0, 0, 1, 0]","[0.010463348, 0.006398678, 0.46447825, 0.23118...","[0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, ...","[0.018872663, 0.012409605, 0.9406123, 0.918916..."


In [90]:
loss_mean = [nn.BCELoss()(torch.tensor(test_df['out_mean'][i]), torch.tensor(test_df['tar'][i])) for i in test_df.index]
np.mean(loss_mean)

0.2171346

In [91]:
df_mean = test_df.drop_duplicates('id')

In [92]:
out_mean = np.vstack([df_mean['out_mean'][i]>=0.5 for i in df_mean.index])
targets = np.vstack([df_mean['tar'][i] for i in df_mean.index])
#targets = dev_tar
accuracy = metrics.accuracy_score(targets, out_mean)
f1_score_micro = metrics.f1_score(targets, out_mean, average='micro')
f1_score_macro = metrics.f1_score(targets, out_mean, average='macro')

print(f"F1 Score (Micro) = {f1_score_micro}")
print(f"F1 Score (Macro) = {f1_score_macro}")

F1 Score (Micro) = 0.6972617743702081
F1 Score (Macro) = 0.6797622228347315


In [93]:
print(classification_report(targets, out_mean, target_names=icd_classes_50, digits=4))

              precision    recall  f1-score   support

      250.00     0.6152    0.6529    0.6335      1092
       272.4     0.7310    0.6553    0.6911      1236
       401.9     0.8190    0.6652    0.7341      2476
      414.01     0.8308    0.7736    0.8012      1396
      427.31     0.7305    0.7851    0.7568      1633
       428.0     0.6037    0.8815    0.7166      1654
      518.81     0.5875    0.7552    0.6609      1111
      530.81     0.8315    0.4927    0.6187       751
       584.9     0.6132    0.7146    0.6600      1300
       599.0     0.6209    0.4542    0.5246       927

   micro avg     0.6913    0.7034    0.6973     13576
   macro avg     0.6983    0.6830    0.6798     13576
weighted avg     0.7075    0.7034    0.6958     13576
 samples avg     0.6653    0.6977    0.6520     13576



In [94]:
ruc_auc_score_micro = metrics.roc_auc_score(targets, out_mean, average='micro')
ruc_auc_score_macro = metrics.roc_auc_score(targets, out_mean, average='macro')

print(f"RUC AUC Score (Micro) = {ruc_auc_score_micro}")
print(f"RUC AUC Score (Macro) = {ruc_auc_score_macro}")

RUC AUC Score (Micro) = 0.7958680378024571
RUC AUC Score (Macro) = 0.7837156877134068
